<a href="https://colab.research.google.com/github/shaankhurshid/bmi707/blob/pre-processing/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Copying the data from dropbox


In [0]:
# Importing the data
train_controls = pd.read_csv('train_controls_transformed.csv?dl=0')
train_cases = pd.read_csv('train_cases_transformed.csv?dl=0')
holdout_controls = pd.read_csv('holdout_controls_transformed.csv?dl=0')
holdout_cases = pd.read_csv('holdout_cases_transformed.csv?dl=0')

In [0]:
# Combine cases and controls
train_data = pd.concat([train_controls, train_cases])

In [16]:
# Check sizes make sense
print(train_controls.shape)
print(train_cases.shape)
print(train_data.shape)

(693, 112321)
(172, 112321)
(865, 222527)


In [0]:
# Training the RNN for binary classification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, LSTM

num_timesteps = train_data.shape[1]

input = Input(shape=(num_timesteps, ))
x = Embedding(input_dim = vocab_size, output_dim = 100, name='embedding')(input)
x = LSTM(units = 128)(x)
x = Dense(2, activation='sigmoid')(x) # 2 because this is a binary classification problem

model = Model(inputs=input, outputs=x)